# Introduction

This notebook is a simple interactive model of emissions at the Mischabel and Windgällen huts.

The intent is to allow us explore different scenarios and to estimate the impact of possible interventions (i.e., is a given change worthwhile?).

The model focuses on food consumption, transportation (helicopter), and, where available, possibly avoidable fuel consumption (e.g., use of the generator at Mischabelhütte). It assumes that most electricity for lighting (and heating/dehumidification in winter) comes from renewables [CC 240126].

For now the model ignores fuel used for cooking because I could not find data. This consumption is unavoidable, but it could be an important part of the denominator when measuring the overall percentage impact of planned emissions reductions.

# Data sources

The model in this notebook is based on the following data and sources.

## Hut visitation data

| Year | Hut        | Overnights | Meal Sales (k CHF) | Source               |
|------|------------|------------|--------------------|----------------------|
| 2021 | Mischabel  | 2000       | n/a                | GV 220506            |
| 2021 | Windgällen | 2700       | >300               | GV 220506            |
| 2022 | Mischabel  | 3100       | n/a                | GV 230616, CC 221026 |
| 2022 | Windgällen | 3450       | 400 (?)            | CC 221026, CC 230930 |
| 2023 | Mischabel  | 3200       | n/a                | CC 230930, CC 240126 |
| 2023 | Windgällen | 3750       | n/a                | CC 230930, CC 240126 |

## Other notes about hut operation

* Fuel use
  * 900 CHF for generator fuel at Mischabel in 2023 [CC 230930]
* Heli operations
  * ~12 heli flights/season (1/week) to Mischabel [Alicia Köster, communication with Claudine Blaser]
  * Apparently each heli flight flies only ~100 kg of fresh vegetables and meat to the hut. The limiting factor appears to be
    storage/refrigeration space in the hut.
* Vegetarian meals
  * Annina and Wisi at Windgällenhütte already serve exclusively vegetarian meals 3-4 days/week [Alicia Köster]

## Food and drink-related emissions

Data on food-related emissions across the supply chain is from [Our World in Data](https://ourworldindata.org):
* https://ourworldindata.org/environmental-impacts-of-food
* https://ourworldindata.org/grapher/food-emissions-supply-chain
which is based on [Poore and Nemecek, 2018](https://doi.org/10.1126/science.aaq0216), the large meta-analysis of food systems to date.

Additional data for bottled drinks comes from [Carbon Cloud](https://carboncloud.com) and [Da Silva, 2022](https://doi.org/10.1016/j.clcb.2022.100021). The impact of different containers (kegs, bottles, etc.) is based on [Bowler et al., 2014](https://doi.org/10.1016/j.jclepro.2023.140181) and [Cimini and Moresi, 2016](https://doi.org/10.1016/j.jclepro.2015.06.063). The authors provide a range of estimates: I have chosen middle-of-the-road values. Differences in life-cycle emissions for different packaging types are (for now) not incorporated further into transportation emissions (i.e., they do not affect heli emissions estimates).

## Transportation emissions

Data on helicopter performance comes from [Air Zermatt](https://air-zermatt.ch) and the Eurocopter [technical data sheet](https://www.aviatorsdatabase.com/wp-content/uploads/2013/07/AS350-B3.pdf)

Fuel emissions factors are from the [GHG Protocol](https://ghgprotocol.org/sites/default/files/Emission_Factors_from_Cross_Sector_Tools_March_2017.xlsx).


In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Model parameters

# Heli transport
heli = {
    'fuel_burn_rate_kg_per_h': 150,
    'flight_duration_h': 0.33,  # Flight duration per round trip
    'emission_factor_kg_co2_per_kg': 3.1 # https://ghgprotocol.org/sites/default/files/Emission_Factors_from_Cross_Sector_Tools_March_2017.xlsx
}

# Diesel generators
generator = {
    'emission_factor_kg_co2_per_l': 3.2 # https://ghgprotocol.org/sites/default/files/Emission_Factors_from_Cross_Sector_Tools_March_2017.xlsx
}

def heli_emissions(num_flights):
    return num_flights * heli['fuel_burn_rate_kg_per_h'] * heli['flight_duration_h'] * heli['emission_factor_kg_co2_per_kg']

def generator_emissions(liters_per_season):
    return liters_per_season * generator['emission_factor_kg_co2_per_l']


In [71]:
# Food emissions data, all from https://ourworldindata.org/environmental-impacts-of-food

"""
food_kg_co2 = {
    # [ per kg, per 100g protein, per 1000 kcal ], for commodities
    "beef": [99, 50, 36],
    "cheese": [24, 11, 6.2],
    "chicken": [10, 5.7, 5.3],
    "grains": [1.6, 1.3, 0.6],
    "lamb": [40, 20, 12],
    "legumes": [1.8, 0.8, 0.5],
    "pork": [12, 7.6, 5.2],
    "tofu": [3.2, 2.0, 1.2],
    "vegetables": [0.9, 4.0, 3.0],
}
"""

food_types = {
    "beef": {
        "Beef burger": [53.98, 28.81, 21.91],
        "Beef steak": [129.75, 64.19, 68.10],
    },
    "cheese": {
        "Cheddar cheese": [20.75, 8.72, 5.41],
        "Goat's cheese": [19.31, 12.55, 6.22],
        "Parmesan cheese": [24.02, 8.44, 6.37],
    },
    "chicken": {
        "Chicken breast": [9.27, 4.52, 5.83],
    },
    "eggs": [4.44, 3.14, 2.77],
    "grains": {
        "Bread": [0.88, 1.44, 0.41],
        "Muesli": [2.27, 2.35, 0.71],
        "Pasta shells": [1.03, 0.84, 0.29],
        "Rice": [3.93, 9.12, 2.05],
    },
    "lamb": {
        "Lamb (leg)": [30.74, 16.63, 15.18],
        "Lamb chops": [30.90, 17.19, 11.83],
    },
    "legumes": {
        "Beans": [1.37, 0.89, 0.80],
        "Chickpeas": [1.34, 0.96, 0.59],
        "Lentils": [2.54, 2.04, 1.96],
    },
    "meat alternative": {
        "Meat-free burger": [1.02, 1.55, 0.54],
        "Tofu": [1.02, 0.61, 0.54],
    },
    "pork": {
        "Pork chops": [12.16, 7.41, 4.59],
        "Pork sausages": [9.77, 6.91, 3.66],
    },
    "potatoes": [0.5, 2.7, 0.6],
    "vegetables": {
        "Broccoli": [0.90, 3.03, 2.69],
        "Cauliflower": [0.89, 3.56, 2.60],
        "Courgettes": [0.85, 4.50, 3.25],
        "Kale": [0.90, 3.15, 2.64],
        "Spinach": [1.01, 4.08, 3.77],
    },
}

# Compute the average of the values in the food_types list.
food_kg_co2 = {}
for k in food_types:
    if not isinstance(food_types[k], dict):
        food_kg_co2[k] = food_types[k]
    else:
        food_kg_co2[k] = np.mean(list(food_types[k].values()), axis=0)

food_kg_co2_per_kg = {k: v[0] for k, v in food_kg_co2.items()}
food_kg_co2_per_100g_protein = {k: v[1] for k, v in food_kg_co2.items()}
food_kg_co2_per_1000kcal = {k: v[2] for k, v in food_kg_co2.items()}

drink_kg_co2_per_kg = {
    "apple juice bottle": 0.5,
    "almond milk carton": 0.7,
    "beer bottle": 0.8,
    "beer keg": 0.3,
    "cow milk bottle": 3.7,
    "soy milk carton": 0.89,
    "wine": 1.7,
}

def meal_emissions(protein_source):
    # Raise error if protein source is not in the list
    if protein_source not in food_kg_co2_per_100g_protein:
        raise ValueError("Protein source not in list")

    e = food_kg_co2_per_100g_protein[protein_source] * 0.6 # 60g of protein from main source

    # Let's add 100 g of grains, 80 g of potatoes, and 100 g of vegetables
    e += food_kg_co2_per_100g_protein["grains"] * 0.1
    e += food_kg_co2_per_100g_protein["potatoes"] * 0.08
    e += food_kg_co2_per_100g_protein["vegetables"] * 0.1
    return e


In [72]:
huts = {
    "mischabel": {
        "visitors_day": 0,
        "visitors_overnight": 3200,
        # Assuming diesel, and based on 900 CHF per season and 1.8 CHF per liter
        "generator_fuel_per_season_l": 500,
        "heli_resupplies_per_season": 12,
    },
    "windgällen": {
        "visitors_day": 7000, # Based on 300K CHF gastro turnover, 30 CHF per visitor, minus overnights
        "visitors_overnight": 3750,
        "generator_fuel_per_season_l": 0,
        "heli_resupplies_per_season": 20, # Guessing, 1/week for 5 months
    }
}

def total_emissions(hut):
    e = 0
    e += heli_emissions(huts[hut]['heli_resupplies_per_season'])
    e += generator_emissions(huts[hut]['generator_fuel_per_season_l'])
    return e

e = total_emissions(hut="mischabel")
print(f"Total CO2 Emissions for Mischabel: {e:.2f} kg CO2")
e = total_emissions(hut="windgällen")
print(f"Total CO2 Emissions for Windgällen: {e:.2f} kg CO2")


Total CO2 Emissions for Mischabel: 3441.40 kg CO2
Total CO2 Emissions for Windgällen: 3069.00 kg CO2


In [73]:
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

# Interactive function to calculate total emissions based on user input
def interactive_emissions(mischabel_heli=12, windgallen_heli=20, mischabel_fuel_l=500):
    huts["mischabel"]["heli_resupplies_per_season"] = mischabel_heli
    huts["windgällen"]["heli_resupplies_per_season"] = windgallen_heli
    huts["mischabel"]["generator_fuel_per_season_l"] = mischabel_fuel_l

    e = total_emissions(hut="mischabel")
    print(f"Total CO2 Emissions for Mischabel: {e:.2f} kg CO2")
    e = total_emissions(hut="windgällen")
    print(f"Total CO2 Emissions for Windgällen: {e:.2f} kg CO2")

# Create sliders for user interaction
interact(interactive_emissions,
         mischabel_heli=(1, 20),
         windgallen_heli=(1,20),
         mischabel_fuel_l=(0, 2000, 100)
        )

interactive(children=(IntSlider(value=12, description='mischabel_heli', max=20, min=1), IntSlider(value=20, de…

<function __main__.interactive_emissions(mischabel_heli=12, windgallen_heli=20, mischabel_fuel_l=500)>